## Code that writes code!

We all need this.

>
> _metaprogramming (noun)_
>
> (programming) Writing computer programs that write or manipulate other programs (or themselves) as their data...

## Homoiconicity

Code in language X can also be treated as data in language X.

X =
- Scheme
- Lisp
- Julia
- Prolog
- Wolfram Language
- ...

## Code as data ("quoting")

In [75]:
gcd(32,24)

8

In [18]:
ex = :(gcd(32,24))

:(gcd(32, 24))

In [19]:
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol gcd
    2: Int64 32
    3: Int64 24


## Code as data ("quoting")

In [1]:
ex = :(gcd(32,24))

:(gcd(32, 24))

In [22]:
typeof(ex)

Expr

In [21]:
ex.head

:call

In [20]:
ex.args

3-element Array{Any,1}:
   :gcd
 32    
 24    

## Infix

In Sceme everything follows this convention:

```scm
(gcd 32 24)
(+ 32 24)
(* (gcd 32 24) (lcm 32 24))
```

In Julia we write

```julia
gcd(32, 24)
32 + 24
gcd(32, 24) * lcm(32, 24)
```

But Julia is secretly a lisp...

In [23]:
dump(:(32 + 24))

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Int64 32
    3: Int64 24


## Different ways to quote

In [15]:
Expr(:call, :gcd, 32, 24)

:(gcd(32, 24))

In [18]:
ex = :(gcd(32,24))

:(gcd(32, 24))

In [20]:
quote
        g = gcd(32, 24)
        l = lcm(32, 24)
        g*l == 32 * 24
end

quote
    #= In[20]:2 =#
    g = gcd(32, 24)
    #= In[20]:3 =#
    l = lcm(32, 24)
    #= In[20]:4 =#
    g * l == 32 * 24
end

## Manipulating `Expr`

Just like you'd manipulate anything else in Julia.

In [27]:
ex = :(1 - 2)

# Create a new expr with arguments flipped
Expr(:call, ex.args[1], ex.args[3], ex.args[2])

:(2 - 1)

In [29]:
# Can also mutate the arguments

ex.args[3], ex.args[2] = ex.args[2], ex.args[3]

ex

:(2 - 1)

## "Splicing" one expression into another

In [24]:
ex = :(gcd(32, 24))
ex1 = Expr(:call, :+, 1, ex)

:(1 + gcd(32, 24))

In [25]:
ex = :(gcd(32, 24))
ex1 = :(1 + $ex)

:(1 + gcd(32, 24))

In [26]:
ex = :(gcd(32, 24))
ex1 = quote
    1 + $ex
end

quote
    #= In[26]:3 =#
    1 + gcd(32, 24)
end

## Evaluating code-data (aka quoted code aka `Expr` aka Expression)

`eval` function

In [82]:
# Evaluate the expression
ex = :(gcd(32,24))

eval(ex)

8

In [5]:
ex1 = Expr(:call, :+, 1, ex)

eval(ex1)

:(1 + gcd(32, 24))

9

In [40]:
eval(quote
        sqrt($ex1)
end)

3.0

## `@eval`

It's just eval with easier syntax! "syntactic sugar"

In [41]:
# "Splice" it into another expression

@eval begin
        sqrt($ex1)
end

3.0

## Symptom: repititive code

In [43]:
using Interact

In [42]:

function log_info(txt)
    display(node(:div, className="alert alert-info", role="alert", txt))
end

function log_warn(txt)
    display(node(:div, className="alert alert-warning", role="alert", txt))
end

function log_error(txt)
    display(node(:div, className="alert alert-danger", role="alert", txt))
end
log_info(md"**Heads up!** It's not super important!")
log_warn(md"**Warning!** Snow storm alert! Stay warm!")
log_error(md"**Error!** Out of bounds.")

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/e8f953a4abf5e4095b83e225fc5e11254819fa44-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/e8f953a4abf5e4095b83e225fc5e11254819fa44-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

(div { role="alert" className="alert alert-info" }
  **Heads up!** It's not super important!
)

(div { role="alert" className="alert alert-warning" }
  **Warning!** Snow storm alert! Stay warm!
)

(div { role="alert" className="alert alert-danger" }
  **Error!** Out of bounds.
)

## Remedy 1: abstract common parts into a function

In [58]:
function _log(class, txt)
    display(node(:div, className="alert alert-$class", role="alert", txt))
end

log_info(txt) = _log("info", txt)
log_info(txt) = _log("info", txt)
log_info(txt) = _log("info", txt)

log_info(md"**Heads up!** It's not super important!")
log_warn(md"**Warning!** Snow storm alert! Stay warm!")
log_error(md"**Error!** Out of bounds.")

(div { role="alert" className="alert alert-info" }
  **Heads up!** It's not super important!
)

(div { role="alert" className="alert alert-warning" }
  **Warning!** Snow storm alert! Stay warm!
)

(div { role="alert" className="alert alert-danger" }
  **Error!** Out of bounds.
)

## Remedy 2: Generate the repetitive functions!

In [45]:
for (suffix, class) in [(:info, "info"), (:warn, "warning"), (:error, "danger")]
    fn = Symbol("log_$suffix")
    ex = quote
        function $(fn)(txt)
            display(node(:div, className="alert alert-" * $class, role="alert", txt))
        end
    end
    println(ex)
    eval(ex)
end

log_info(md"**Heads up!** It's not super important!")
log_warn(md"**Warning!** Snow storm alert! Stay warm!")
log_error(md"**Error!** Out of bounds.")

(div { role="alert" className="alert alert-info" }
  **Heads up!** It's not super important!
)

(div { role="alert" className="alert alert-warning" }
  **Warning!** Snow storm alert! Stay warm!
)

(div { role="alert" className="alert alert-danger" }
  **Error!** Out of bounds.
)

begin
    #= In[45]:4 =#
    function log_info(txt)
        #= In[45]:5 =#
        display(node(:div, className="alert alert-" * "info", role="alert", txt))
    end
end
begin
    #= In[45]:4 =#
    function log_warn(txt)
        #= In[45]:5 =#
        display(node(:div, className="alert alert-" * "warning", role="alert", txt))
    end
end
begin
    #= In[45]:4 =#
    function log_error(txt)
        #= In[45]:5 =#
        display(node(:div, className="alert alert-" * "danger", role="alert", txt))
    end
end



## Examples from Julia's source code

`convert(T, x::S)` is the standard way to convert `x` to `T`

**Task:** Define all conversions between `Week`, `Day`, `Hour`, `Minute`, `Second`, `Millisecond`, `Microsecond`, `Nanosecond`

8x8 = 64 methods!

In [49]:
using Dates
# from https://github.com/JuliaLang/julia/blob/1d3c371636e159cb47e68783e8d6ac90feaaace0/stdlib/Dates/src/periods.jl#L395-L418
const fixedperiod_conversions = [(:Week, 7), (:Day, 24), (:Hour, 60), (:Minute, 60), (:Second, 1000),
                                 (:Millisecond, 1000), (:Microsecond, 1000), (:Nanosecond, 1)]
for i = 1:length(fixedperiod_conversions)
    T, n = fixedperiod_conversions[i]
    N = Int64(1)
    for j = (i - 1):-1:1 # to less-precise periods
        Tc, nc = fixedperiod_conversions[j]
        N *= nc
        vmax = typemax(Int64) ÷ N
        vmin = typemin(Int64) ÷ N
        @eval function Base.convert(::Type{$T}, x::$Tc)
            $vmin ≤ value(x) ≤ $vmax || throw(InexactError(:convert, $T, x))
            return $T(value(x) * $N)
        end
    end
    N = n
    for j = (i + 1):length(fixedperiod_conversions) # to more-precise periods
        Tc, nc = fixedperiod_conversions[j]
        @eval Base.convert(::Type{$T}, x::$Tc) = $T(divexact(value(x), $N))
        N *= nc
    end
end

## Macro

### General idea:

```julia
@foo ex1 ex2
```

is re-written as per:

```julia
macro foo(ex1, ex2)
    ...
end
```

which _returns_ the code to execute.

### When are macros re-written?

Right after parsing the code, Julia first re-writes macro calls to their corresponding expressions, it then executes the re-written code!

## Macro example - @flip

In [50]:
macro flip(ex)
    Expr(ex.head, ex.args[1], ex.args[3], ex.args[2])
end

@flip [1,2,3] * [2,3,4]'

# [1,2,3] * [2,3,4]'

20

`@macroexpand` is a macro that expands a macro call!

In [102]:
@macroexpand @flip [1,2,3] * [2,3,4]'

:(([2, 3, 4])' * [1, 2, 3])

## Macro example from Julia - @time

In [103]:
@time peakflops()

  0.379349 seconds (14 allocations: 61.188 MiB, 8.93% gc time)


4.346710801284569e10

In [111]:
@which @time peakflops()

@time(__source__::LineNumberNode, __module__::Module, ex) in Base at util.jl:153

In [56]:
ex1 = @macroexpand @time peakflops()

quote
    #= util.jl:154 =#
    local #74#stats = (Base.gc_num)()
    #= util.jl:155 =#
    local #76#elapsedtime = (Base.time_ns)()
    #= util.jl:156 =#
    local #75#val = peakflops()
    #= util.jl:157 =#
    #76#elapsedtime = (Base.time_ns)() - #76#elapsedtime
    #= util.jl:158 =#
    local #77#diff = (Base.GC_Diff)((Base.gc_num)(), #74#stats)
    #= util.jl:159 =#
    (Base.time_print)(#76#elapsedtime, (#77#diff).allocd, (#77#diff).total_time, (Base.gc_alloc_count)(#77#diff))
    #= util.jl:161 =#
    (Base.println)()
    #= util.jl:162 =#
    #75#val
end

## MacroTools.prettify removes junk

In [57]:
using MacroTools

prettify(ex1)

quote
    local alpaca = (Base.gc_num)()
    local hummingbird = (Base.time_ns)()
    local panther = peakflops()
    hummingbird = (Base.time_ns)() - hummingbird
    local seal = (Base.GC_Diff)((Base.gc_num)(), alpaca)
    (Base.time_print)(hummingbird, seal.allocd, seal.total_time, (Base.gc_alloc_count)(seal))
    (Base.println)()
    panther
end

## Excercise: Differentiation!

In [144]:
repexpr(ex::Symbol, sym, repex) = ex == sym ? repex : ex
function repexpr(ex, sym, repex)
    Expr(ex.head, repexpr.(ex.args, sym, (repex,))...)
end

function diffexpr(ex, wrt)
    if ex isa Number
        return 0.0
    elseif ex isa Symbol
        ex == wrt ? 1.0 : 0.0
    end
        
    @assert ex.head == :call
    fn = ex.args[1]

    # the derivative of f(g(x)) = f'(g(x))g'(x)
    # so if a call expr contains another call! intercept!
    if ex.args[2] isa Expr && ex.head == :call && length(ex.args) == 2
        g = ex.args[2]
        f′ = diffexpr(Expr(:call, ex.args[1], wrt), wrt)
        # replace x with g(x)
        f′ = repexpr(f′, wrt, g)
        g′ = diffexpr(g)
        return :($f′ * $g′)
    end

    if fn == :sin
        return Expr(:call, :cos, ex.args[2])
    elseif fn == :cos
        Expr(:call, :-, Expr(:call, :sin, ex.args[2]))
    elseif fn == :exp
        # ...Fill this!
    elseif fn == :^
        @assert ex.args[3] isa Number
        # ...
    elseif fn == :+
        # addition rule    (f + g)’ = f’ + g’
    elseif fn == :*
        # product rule!    (f g)’ = f g’ + f’ g
    elseif fn == :/
        # quotient rule!!  (f/g)’ = (f’ g − g’ f )/(g^2)
    end
end

diffexpr (generic function with 2 methods)

In [136]:
# This should already work!
diffexpr(:(sin(x)), :x) |> println
diffexpr(:(cos(x)), :x) |> println
diffexpr(:(cos(sin(x))), :x) |> println

cos(x)
-(sin(x))
-(sin(sin(x))) * cos(x)


## Make thes work!

In [137]:
# Task 0:

diffexpr(:(exp(x)), :x)

In [109]:
# Task 1:

diffexpr(:(x^3), :x)

In [110]:
# Task 2:

diffexpr(:(sin(x) + cos(x)), :x)

In [ ]:
# Task 3:

diffexpr(:(sin(x) * cos(x)), :x)

In [ ]:
# Task 4:

diffexpr(:(sin(x) / cos(x)), :x)

## Do these expressions work (composition)?

In [111]:
diffexpr(:(sin(cos(x)) + cos(x)), :x)

In [112]:
diffexpr(:(sin(x^2) + cos(x^2)), :x)

In [113]:
diffexpr(:(exp(x^2)), :x)

ErrorException: type Nothing has no field head

## Write the macro

Recall:

```julia
@foo ex1 ex2
```

is re-written as per:

```julia
macro foo(ex1, ex2)
    ...
end
```


In [121]:
macro diff(ex, wrt)
    diffexpr(ex, wrt)
end

@diff (macro with 1 method)

In [138]:
dsin(x) = @diff sin(x) x

dsin (generic function with 1 method)

In [139]:
dsin(pi/2)

6.123233995736766e-17

# `esc` -- treat variables as coming from outside the macro!

In [140]:
macro diff(ex, wrt)
    esc(diffexpr(ex, wrt))
end

@diff (macro with 1 method)

In [141]:
dsin(x) = @diff sin(x) x

dsin (generic function with 1 method)

In [142]:
dsin(pi/2)

6.123233995736766e-17